In [ ]:
from datetime import date
from data_retrieval.google_analytics_api_retrieval import GoogleAnalyticsApiRetrieval
from dtos.date_range_dto import DateRangeDto
from dtos.google_analytics_filter_clause_dto import GoogleAnalyticsFilterClause
from dtos.page_dto import PageDto
from helpers.enums import GoogleApiVersion, GoogleAuthenticationMethod
from helpers.settings_helper import SettingsHelper
from dtos.google_analytics_request_config_dto import GoogleAnalyticsRequestConfig

import pandas as pd

credentials_file_path = './credentials/credentials.json'
token_file_path = './credentials/token.json'
settings_helper = SettingsHelper()

def get_dimension_metrics(module: str, version: int):
    print('Module ', module)
    print('version ', version)
    df = pd.read_csv('./settings/google_api_metrics.csv')
    row = df[(df['module'] == module) & (df['version'] == version)]
    if len(row) == 0:
        return [], []
    dimensions = [d.strip() for d in row['dimensions'].values[0].split(",")]
    metrics = [m.strip() for m in row['metrics'].values[0].split(",")]
    return dimensions, metrics


google_analytics_api = GoogleAnalyticsApiRetrieval(
            google_authentication_method=GoogleAuthenticationMethod.OAUTH,
            oauth_credentials_filepath=credentials_file_path,
            oauth_token_filepath=token_file_path)

filter_clause = GoogleAnalyticsFilterClause()
filter_clause.set_council_name('burnside')
filter_clause.set_date_range(DateRangeDto(start_date=date(2020,4,1), end_date=date(2024,5,1)))
page_size = SettingsHelper().get_google_analytics_page_size()
filter_clause.set_page_dto(PageDto(page_size, None))

version = GoogleApiVersion.VERSION_4
filter_clause.set_api_version(version)

module = 'Sessions by Landing Page'

dimensions, metrics = get_dimension_metrics(module, version.value)
print('dimensions ', dimensions)
print('metrics ', metrics)

request_config=GoogleAnalyticsRequestConfig(dimensions, metrics)

page_views_and_sessions_df = google_analytics_api.get_data_as_df(request_config, filter_clause)
page_views_and_sessions_df.head()

In [ ]:
sessions_by_landing_page_df = google_analytics_api.get_sessions_by_landing_page(filter_clause)
sessions_by_landing_page_df.head()